In [1]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Mounted at /content/drive


## 1. 전처리

In [3]:
train["prompt"].isna().sum()

0

In [4]:
# train = train[train["prompt"].isna()==False]

In [5]:
train["answer"].isna().sum()

1

In [6]:
train = train[train["answer"].isna()==False]

In [7]:
train["AI"].isna().sum()

1

In [8]:
train = train[train["AI"].isna()==False]

In [9]:
test.isna().sum()

ID        0
prompt    0
answer    0
dtype: int64

In [10]:
train

,ID,prompt,answer,AI
0,690.0,What is the future of AI?,The future of Artificial Intelligence (AI) is ...,1.0
1,304.0,What is your biggest challenge in your career?,My biggest challenge in my career is balancing...,0.0
2,63.0,What is the tallest mountain in the world?,The tallest mountain in the world is Mount Eve...,1.0
3,894.0,What are the best 5 super cars?,McLaren 720S.\r\nFerrari 296 GTB.\r\nAudi R8.\...,0.0
4,796.0,What is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,What is the most important thing in a family?,The most important thing in a family for me is...,0.0
744,895.0,What are the best machine learning Models?,Linear regression.\r\nLogistic regression\r\nD...,0.0
745,963.0,How can I chop onions without crying?,The best thing to do is very simple: use a ver...,0.0
746,156.0,What is the best way to stay healthy?,The best way to stay healthy varies from perso...,0.0


In [11]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7410c053df5e82e12217248bd0fbdb2b367e9f9fc65a8e8b7891fcaabb27e05a
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [12]:
import numpy as np

In [13]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
import emoji
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x

In [14]:
train_text = train.copy()
train_text["prompt"]=train["prompt"].apply(lambda x: dataPreprocessing(x))
train_text["answer"]=train["answer"].apply(lambda x: dataPreprocessing(x))

In [15]:
test_text = test.copy()
test_text["prompt"]=test["prompt"].apply(lambda x: dataPreprocessing(x))
test_text["answer"]=test["answer"].apply(lambda x: dataPreprocessing(x))

In [16]:
train_text

,ID,prompt,answer,AI
0,690.0,what is the future of ai,the future of artificial intelligence ai is bo...,1.0
1,304.0,what is your biggest challenge in your career,my biggest challenge in my career is balancing...,0.0
2,63.0,what is the tallest mountain in the world,the tallest mountain in the world is mount eve...,1.0
3,894.0,what are the best super cars,mclaren\nferrari gtb\r\naudi\nmaserati mc\r\n...,0.0
4,796.0,what is deep learning,a type of machine learning based on artificial...,0.0
...,...,...,...,...
743,283.0,what is the most important thing infamily,the most important thing infamily for me is lo...,0.0
744,895.0,what are the best machine learning models,linear regression\r\nlogistic regression\r\nde...,0.0
745,963.0,how canchop onions without crying,the best thing to do is very simple usevery sh...,0.0
746,156.0,what is the best way to stay healthy,the best way to stay healthy varies from perso...,0.0


In [17]:
from sklearn.model_selection import train_test_split

X = train_text.drop(['ID'], axis=1)
y = train['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
pd.DataFrame(X_train, columns=['prompt', 'answer'])

,prompt,answer
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt
164,what is the name of the largest underwater cav...,sac actun cave system
712,what is the name of the gas that makes up the ...,the gas that makes up the majority of the eart...
265,what is the name of the largest underwater cav...,after doinglot of researchfound that sac actun...
250,what is the most important factor in makingdec...,the most important factor in makingdecision fo...
...,...,...
71,what is your biggest strength,my biggest strength is my ability to communica...
106,what is the currency of australia,australian dollar
270,what is the best way to stay positive,the best way to stay positive in my opinion is...
435,what is the name of the largest underwater cav...,sac actun cave system


In [19]:
pd.DataFrame(y_train, columns=['AI'])

,AI
70,1.0
164,0.0
712,0.0
265,0.0
250,0.0
...,...
71,0.0
106,0.0
270,0.0
435,0.0


In [20]:
train_data = pd.concat([pd.DataFrame(X_train, columns=['prompt', 'answer']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data

,prompt,answer,AI
70,what is the name of the unit used to measure p...,the unit used to measure power is the watt,1.0
164,what is the name of the largest underwater cav...,sac actun cave system,0.0
712,what is the name of the gas that makes up the ...,the gas that makes up the majority of the eart...,0.0
265,what is the name of the largest underwater cav...,after doinglot of researchfound that sac actun...,0.0
250,what is the most important factor in makingdec...,the most important factor in makingdecision fo...,0.0
...,...,...,...
71,what is your biggest strength,my biggest strength is my ability to communica...,0.0
106,what is the currency of australia,australian dollar,0.0
270,what is the best way to stay positive,the best way to stay positive in my opinion is...,0.0
435,what is the name of the largest underwater cav...,sac actun cave system,0.0


In [21]:
test_data = pd.concat([pd.DataFrame(X_test, columns=['prompt', 'answer']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
test_data

,prompt,answer,AI
208,what is the most important lesson you have lea...,the most important lessonhave learned in life ...,0.0
259,what is your biggest challenge in life,my biggest challenge in life is balancing my w...,0.0
97,where to put code documentation,some use other systems takelook at pythons sp...,0.0
148,what is the key to success in the culinary arts,the key to success in the culinary arts is cre...,0.0
395,what is your take on the current state of the ...,the current state of the world is complex and ...,1.0
...,...,...,...
554,what is the best way to stay organized,the best way to stay organized varies from per...,0.0
69,what is the name of the largest living structu...,the largest living structure in the world is t...,1.0
522,what is the meaning of life,the meaning of life is subjective and can vary...,0.0
131,what is natural language processing nlp,nlp is an interdisciplinary subfield of lingu...,0.0


# 2. 모델링

In [23]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

In [24]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=10, objective='val_accuracy', seed=32
)  

In [25]:
# 모델 저장 경로
model_dir = PATH+"model/autokeras_qa_val_acc_230304/"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# 체크포인트 파일 저장 경로
checkpoint_path = model_dir+"checkpoints_model_autokeras_230304.ckpt"

In [26]:
callbacks = [
tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True, verbose=1, save_best_only=True
),
tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.5, patience=5, min_lr=0.0001
),
tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode='max', patience=10, verbose=1),
]

In [35]:
x_train = np.hstack((train_data.drop(['AI'], axis=1), np.zeros((train_data.drop(['AI'], axis=1).shape[0], 1))))
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1])).astype('float32')  # (batch_size, 1, 3)

ValueError: ignored

In [32]:
x_train.shape

(596, 1, 3)

In [34]:
x_train

array([[['what is the name of the unit used to measure power',
         'the unit used to measure power is the watt', 0.0]],

       [['what is the name of the largest underwater cave system in the world',
         'sac actun cave system', 0.0]],

       [['what is the name of the gas that makes up the majority of the earths atmosphere',
         'the gas that makes up the majority of the earths atmosphere is nitrogen',
         0.0]],

       ...,

       [['what is the best way to stay positive',
         'the best way to stay positive in my opinion is to surround myself with positive influences practice gratitude and focus on the present moment',
         0.0]],

       [['what is the name of the largest underwater cave system in the world',
         'sac actun cave system', 0.0]],

       [['what is your opinion on the current state of ai technology',
         'ai technology has advanced significantly in recent years and is being increasingly integrated into various industries and 

In [33]:
# Feed the text classifier with training data.
clf.fit(x_train, train_data['AI'].values, callbacks=callbacks, validation_split=0.2, epochs=30)

ValueError: ignored

# 3. 성능 평가

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Predict with the best model.
predicted_y = clf.predict(X_test.values)
# Evaluate the best model with testing data.
print(classification_report(y_test.values, predicted_y))

5/5 [==============================] - 1s 194ms/step
              precision    recall  f1-score   support

         0.0       0.96      0.87      0.91       104
         1.0       0.75      0.91      0.82        46

    accuracy                           0.88       150
   macro avg       0.85      0.89      0.87       150
weighted avg       0.89      0.88      0.88       150



In [ ]:
# model_to_use = predictor.get_model_best()
model_pred = clf.predict(test_text.values)

8/8 [==============================] - 2s 191ms/step


In [ ]:
submission['Category'] = model_pred

In [ ]:
submission = submission.astype({'Category':'int64'})

In [ ]:
submission

,Id,Category
0,710,1
1,487,0
2,136,1
3,44,1
4,627,0
...,...,...
244,702,1
245,500,1
246,818,0
247,584,0


In [ ]:
submission.to_csv(PATH+'submit/autokeras_QA_val_acc_submission_230304.csv', index=False)